Using a DTS Resolver
==================

*This example is powered up thanks to the API available at Alpheios.net*


In [1]:
# Just importing the parent module, do not change this if you execute the code
# from the git repository
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)


## Create the resolver

With the following line we create the resolver :

In [2]:
from MyCapytain.resolvers.dts.api_v1 import HttpDtsResolver

resolver = HttpDtsResolver("http://texts.alpheios.net/api/dts")

## Require metadata : let's visit the catalog

The following code is gonna find each text that is readable by Alpheios

In [3]:
# We get the root collection
root = resolver.getMetadata()
# Then we retrieve dynamically all the readableDescendants : it browse automatically the API until
# it does not have seen any missing texts: be careful with this one on huge repositories
readable_collections = root.readableDescendants
print("We found %s collections that can be parsed" % len(readable_collections))

We found 15 collections that can be parsed


### Printing the full tree

In [4]:
# Note that we could also see and make a tree of the catalog.
# If you are not familiar with recursivity, the next lines might be a bit complicated
def show_tree(collection, char_number=1):
    for subcollection_id, subcollection in collection.children.items():
        print(char_number*"--" + " " + subcollection.id)
        show_tree(subcollection, char_number+1)

print(root.id)
show_tree(root)

default
-- urn:alpheios:latinLit
---- urn:cts:latinLit:phi0472
------ urn:cts:latinLit:phi0472.phi001
-------- urn:cts:latinLit:phi0472.phi001.alpheios-text-lat1
---- urn:cts:latinLit:phi0620
------ urn:cts:latinLit:phi0620.phi001
-------- urn:cts:latinLit:phi0620.phi001.alpheios-text-lat1
---- urn:cts:latinLit:phi0959
------ urn:cts:latinLit:phi0959.phi006
-------- urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1
-- urn:alpheios:greekLit
---- urn:cts:greekLit:tlg0085
------ urn:cts:greekLit:tlg0085.tlg004
-------- urn:cts:greekLit:tlg0085.tlg004.alpheios-text-grc1
------ urn:cts:greekLit:tlg0085.tlg005
-------- urn:cts:greekLit:tlg0085.tlg005.alpheios-text-grc1
------ urn:cts:greekLit:tlg0085.tlg006
-------- urn:cts:greekLit:tlg0085.tlg006.alpheios-text-grc1
------ urn:cts:greekLit:tlg0085.tlg003
-------- urn:cts:greekLit:tlg0085.tlg003.alpheios-text-grc1
------ urn:cts:greekLit:tlg0085.tlg001
-------- urn:cts:greekLit:tlg0085.tlg001.alpheios-text-grc1
------ urn:cts:greekLit:tlg008

### Printing details about a specific one

In [5]:
# Let's get a random one !
from random import randint
# The index needs to be between 0 and the number of collections
rand_index = randint(0, len(readable_collections))
collection = readable_collections[rand_index]

# Now let's print information ?
label = collection.get_label()

text_id = collection.id
print("Treaing `"+label+"` with id " + text_id)

Treaing `Metamorphoses` with id urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1


#### What about more detailed informations ? Like the citation scheme ?

In [6]:
def recursive_printing_citation_scheme(citation, char_number=1):
    for subcitation in citation.children:
        print(char_number*"--" + " " + subcitation.name)
        recursive_printing_citation_scheme(subcitation, char_number+1)

print("Maximum citation depth : ", collection.citation.depth)
print("Citation System")
recursive_printing_citation_scheme(collection.citation)

Maximum citation depth :  2
Citation System
-- book
---- line


### Let's get some references !

In [7]:
reffs = resolver.getReffs(collection.id)
print(reffs)
# Nice !

<DtsReferenceSet (<DtsReference <1> [book]>, <DtsReference <2> [book]>, <DtsReference <3> [book]>, <DtsReference <4> [book]>, <DtsReference <5> [book]>, <DtsReference <6> [book]>, <DtsReference <7> [book]>, <DtsReference <8> [book]>, <DtsReference <9> [book]>, <DtsReference <10> [book]>, <DtsReference <11> [book]>, <DtsReference <12> [book]>, <DtsReference <13> [book]>, <DtsReference <14> [book]>, <DtsReference <15> [book]>) level:1, citation:<DtsCitation name(book)>>


### Let's get some random passage !


In [8]:
# Let's get a random one !
from random import randint
# The index needs to be between 0 and the number of collections
rand_index = randint(0, len(reffs)-1)
reff = reffs[rand_index]

passage = resolver.getTextualNode(collection.id, reff)
print(passage.id, passage.reference)

# Let's see the XML here
# For that, we need to get the mimetype right :
from MyCapytain.common.constants import Mimetypes
print(passage.export(Mimetypes.XML.TEI))

urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1 <DtsReference <3> [book]>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text><body n="urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"><div type="edition" xml:lang="lat" n="urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"><div type="textpart" subtype="book" n="3"><milestone n="1" unit="card"/><milestone ed="Magnus" n="Cadmus." unit="tale"/><milestone ed="P" unit="para"/><l n="1">Iamque deus posita fallacis imagine tauri</l><l n="2">se confessus erat Dictaeaque rura tenebat,</l><l n="3">cum pater ignarus Cadmo perquirere raptam</l><l n="4">imperat et poenam, si non invenerit, addit</l><l n="5">exilium, facto pius et sceleratus eodem.</l><milestone ed="P" unit="para"/><l n="6">Orbe pererrato (quis enim deprendere possit</l><l n="7">furta Iovis?) profugus patriamque iramque parentis</l><l n="8">vitat Agenorides Phoe

### (If you are lucky on random) Get sub passage because it's too big

Let's get the first and the last passage of this text shall we ?

In [9]:
first_passage = passage.first
print(first_passage.export(Mimetypes.XML.TEI))

<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text><body n="urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"><div type="edition" xml:lang="lat" n="urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"><div type="textpart" subtype="book" n="3"><l n="1">Iamque deus posita fallacis imagine tauri</l></div></div></body></text></dts:fragment></TEI>


In [10]:
# Still too big ? Then let's get the child of the child passage
first_passage_last_child = first_passage.last
print(first_passage_last_child.export(Mimetypes.XML.TEI))

HTTPError: 404 Client Error: NOT FOUND for url: https://texts.alpheios.net/api/dts/navigation?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&level=1&groupBy=1&ref=3.1